## Aula 03 - Filtragem Baseada em Conteúdo - Exercícios

In [16]:
import pandas as pd
import numpy as np

### Importar base de dados

In [17]:
import wget
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
!tar -xvzf ml-20m-compact.tar.gz

100% [....................................................] 65019041 / 65019041
Saved under ml-20m-compact.tar (1).gz
dataset/
dataset/tags_sample.csv
dataset/._.DS_Store
dataset/.DS_Store
dataset/movies_sample.csv
dataset/._genome-tags.csv
dataset/genome-tags.csv
dataset/._ml-youtube.csv
dataset/ml-youtube.csv
dataset/._genome-scores.csv
dataset/genome-scores.csv
dataset/ratings_sample.csv


In [18]:
movies = pd.read_csv('./dataset/movies_sample.csv')
ratings = pd.read_csv('./dataset/ratings_sample.csv')
df = ratings[['userId', 'movieId', 'rating']]
df = df.merge(movies[['movieId', 'title']])
df

,userId,movieId,rating,title
0,11,7481,5.0,Enemy Mine (1985)
1,11,1046,4.5,Beautiful Thing (1996)
2,11,616,4.0,"Aristocats, The (1970)"
3,11,3535,2.0,American Psycho (2000)
4,11,5669,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,138493,288,5.0,Natural Born Killers (1994)
190617,138493,1748,5.0,Dark City (1998)
190618,138493,616,4.0,"Aristocats, The (1970)"
190619,138493,1597,4.5,Conspiracy Theory (1997)


In [19]:
movies_tags = pd.read_csv('./dataset/tags_sample.csv')
movies_tags.head()

,userId,movieId,tag,timestamp_y
0,279,916,Gregory Peck,1329962459
1,279,916,need to own,1329962471
2,279,916,romantic comedy,1329962476
3,279,916,Rome,1329962490
4,279,916,royalty,1329962474


In [20]:
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.movieId.unique())}
df['userId'] = df['userId'].map(map_users)
df['movieId'] = df['movieId'].map(map_items)
movies_tags['userId'] = movies_tags['userId'].map(map_users)
movies_tags['movieId'] = movies_tags['movieId'].map(map_items)
map_title = {}

for _, row in df.iterrows():
    map_title[row.movieId] = row.title


In [21]:
movies_tags.head()

,userId,movieId,tag,timestamp_y
0,18,34,Gregory Peck,1329962459
1,18,34,need to own,1329962471
2,18,34,romantic comedy,1329962476
3,18,34,Rome,1329962490
4,18,34,royalty,1329962474


### Divisão da base em treino e teste

In [22]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)

***Exercício 01:*** Aplique a filtragem baseada em conteúdo (ItemAttributeKNN do CaseRecommender) com as tags associadas aos filmes da base. Utilize Jaccard como métrica de similaridade e k=5 vizinhos para predição.

Documentação do ItemAttributeKNN: https://github.com/caserec/CaseRecommender/blob/master/caserec/recommenders/rating_prediction/item_attribute_knn.py

In [29]:
from collections import defaultdict
import numpy as np

# dicionário {item: [tags]}
tag_by_item = defaultdict(list)
for _, row in movies_tags.iterrows():
    tag_by_item[row.movieId].append(row.tag)

# similaridade Jaccard
def jaccard(a, b):
    A, B = set(a), set(b)
    return len(A & B) / len(A | B) if A or B else 0

# matriz de similaridade entre itens
n_items = df.movieId.max() + 1
S = np.zeros((n_items, n_items))
for i in range(n_items):
    for j in range(i + 1, n_items):
        S[i, j] = S[j, i] = jaccard(tag_by_item[i], tag_by_item[j])

# predição com k vizinhos
def predict(uid, iid, k=5):
    user_hist = train[train.userId == uid]
    if user_hist.empty:
        return 3.0  # fallback neutro

    # considere apenas vizinhos com similaridade > 0
    sims = [(S[iid, j], r) for j, r in zip(user_hist.movieId, user_hist.rating) if S[iid, j] > 0]
    sims.sort(reverse=True)
    sims = sims[:k]

    if not sims:
        # se não há vizinhos semelhantes, cai para a média do usuário
        return float(user_hist.rating.mean())

    num = sum(s * r for s, r in sims)
    den = sum(s for s, _ in sims)

    return num / den if den != 0 else float(user_hist.rating.mean())

# gera predições para o conjunto de teste
preds = [predict(u, i) for u, i in zip(test.userId, test.movieId)]

# métrica de erro
mae = np.mean(np.abs(test.rating - preds))
print("MAE:", mae)


MAE: 0.8009755521361313


### Preparação para o exercício 2 - Download e extração de metadados multimídia

In [24]:
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-features.tar.gz
! tar -xvzf ml-20m-features.tar.gz

100% [......................................................] 5996435 / 5996435
Saved under ml-20m-features.tar (1).gz
features/
features/._m4infus_max_histogram_300_sn.arq
features/m4infus_max_histogram_300_sn.arq
features/._mm_avg_histogram_100_sn.arq
features/mm_avg_histogram_100_sn.arq
features/._visual_histogram_100_sn.arq
features/visual_histogram_100_sn.arq
features/._visual_histogram_50_sn.arq
features/visual_histogram_50_sn.arq
features/._aural_histogram_50.arq
features/aural_histogram_50.arq
features/._mm_max_histogram_300.arq
features/mm_max_histogram_300.arq
features/._m4infus_max_histogram_50.arq
features/m4infus_max_histogram_50.arq
features/._mm_max_histogram_100.arq
features/mm_max_histogram_100.arq
features/._mm_max_histogram_50_sn.arq
features/mm_max_histogram_50_sn.arq
features/._visual_histogram_100.arq
features/visual_histogram_100.arq
features/._visual_histogram_300.arq
features/visual_histogram_300.arq
features/._aural_histogram_100_sn.arq
features/aural_histogra

In [25]:
import pickle

with open('./features/visual_histogram_50.arq', 'rb') as arq_visualHistograms:
    visualHistograms = pickle.load(arq_visualHistograms)
print('No. movies: ' + str(len(visualHistograms)))
print('Size of each word: ' + str(len(visualHistograms[0])))

No. movies: 433
Size of each word: 50


In [26]:
visualHistograms[0]

array([0.00948992, 0.00237248, 0.00355872, 0.01304864, 0.05338078,
       0.00711744, 0.05456702, 0.00355872, 0.0059312 , 0.00355872,
       0.0059312 , 0.00474496, 0.00711744, 0.04507711, 0.07591934,
       0.00355872, 0.1316726 , 0.05338078, 0.00830368, 0.        ,
       0.01304864, 0.00474496, 0.02253855, 0.00355872, 0.02728351,
       0.00237248, 0.02016607, 0.00830368, 0.0059312 , 0.00237248,
       0.00355872, 0.04033215, 0.01067616, 0.00237248, 0.00830368,
       0.05931198, 0.0059312 , 0.01067616, 0.02965599, 0.01423488,
       0.00711744, 0.02609727, 0.        , 0.00237248, 0.01423488,
       0.03084223, 0.02016607, 0.00118624, 0.08778173, 0.02253855])

***Exercício 02:*** Como visto, o algoritmo ItemAttributeKNN pode ser usado com diferentes tipos de metadados, como gêneros, tags e palavras no geral. Mais do que isso, podemos adaptá-lo também para que a similaridade entre itens seja feita com base em informações multimídia, como imagens, áudio, etc. 

A base de dados utilizada até o momento, ml-20m-compact.tar.gz possui, além das interações de usuários com filmes, uma série de arquivos que contém informações multimídia que foram extraídas dos trailers de cada filme. Esses arquivos estão condensados no zip ml-20m-features.tar.gz, o qual foi feito o download e extraído acima. 

Considere por exemplo o arquivo visual_histogram_50.arq. Ele possui 433 vetores (no. de filmes) de tamanho 50. Podemos pensar que cada vetor desse representa informações visuais (cor, brilho, imagem, etc.) que foram extraídas dos trailers de cada filme. 

Sua tarefa é usar esses vetores de características visuais no cálculo de similaridade entre os filmes, e em seguida, aplicar essas similaridades no algoritmo ItemAttributeKNN para gerar recomendações. 

Dica 1: para calcular a similaridade entre dois vetores pode-se usar o ângulo de cosseno (vide https://en.wikipedia.org/wiki/Cosine_similarity). 

Dica 2: é possível passar para o algoritmo ItemAttributeKNN a matriz de similaridade entre itens, por meio do parâmetro similarity_file=arquivo. Veja em: https://github.com/caserec/CaseRecommender/blob/master/caserec/recommenders/rating_prediction/item_attribute_knn.py

In [ ]:
import numpy as np
from numpy.linalg import norm

# 1) Matriz de características visuais (já carregada como 'visualHistograms')
M = np.array(visualHistograms, dtype=float) 
n_feat_items = M.shape[0]

# 2) Similaridade por cosseno entre itens-com-feature
X = M / np.clip(norm(M, axis=1, keepdims=True), 1e-12, None)
S_visual_small = X @ X.T
np.clip(S_visual_small, -1.0, 1.0, out=S_visual_small)
np.fill_diagonal(S_visual_small, 1.0)

# 3) Embutir S_visual_small no espaço total de itens de df (pode haver tamanhos diferentes)
n_items_total = int(df.movieId.max()) + 1
S_visual = np.zeros((n_items_total, n_items_total), dtype=float)

# usa apenas a interseção de índices válida para os dois lados
m = min(n_feat_items, n_items_total)
S_visual[:m, :m] = S_visual_small[:m, :m]

# 4) Predição com K vizinhos usando S_visual (com proteções)
def predict_visual(uid, iid, k=5):
    user_hist = train[train.userId == uid]
    if user_hist.empty:
        return 3.0  # fallback neutro

    # só considera vizinhos com similaridade > 0
    sims = []
    for j, r in zip(user_hist.movieId, user_hist.rating):
        s = S_visual[iid, j]
        if s > 0:
            sims.append((s, r))

    sims.sort(key=lambda x: x[0], reverse=True)
    sims = sims[:k]

    if not sims:
        # sem vizinhos visuais úteis → média do usuário
        return float(user_hist.rating.mean())

    num = sum(s * r for s, r in sims)
    den = sum(s for s, _ in sims)
    return (num / den) if den != 0 else float(user_hist.rating.mean())

# 5) Avaliação no conjunto de teste
preds_visual = [predict_visual(u, i) for u, i in zip(test.userId, test.movieId)]
mae_visual = np.mean(np.abs(test.rating - preds_visual))
rmse_visual = float(np.sqrt(np.mean((test.rating - preds_visual) ** 2)))
print(f"MAE (visual): {mae_visual:.6f}")
print(f"RMSE (visual): {rmse_visual:.6f}")

MAE (visual): 0.805868
RMSE (visual): 1.036540


***Exercício 03:*** Implementar uma função que retorna a probabilidade de um item ser relevante para um usuário, considerando os gêneros dos filmes. Utilize métodos probabilísticos. 
- A partir do conjunto de treinamento, obter todas as interações do usuário u.
- Rotular as notas desse usuário como: item relevante se nota >=3 e não relevante se nota < 3. 
- Dado um item do conjunto de teste, aplicar o Teorema de Bayes com suavização de Laplace. Utilizar os gêneros associados. 
- Retornar se o item é ou não relevante, e em seguida, comparar o resultado com a nota real que esse usuário deu para o item (disponível no conjunto de teste).

In [33]:
from collections import Counter
import numpy as np
import pandas as pd

# 1) Mapeia gêneros para os IDs INTERNOS (mesmo espaço de df/train/test)
movies_mapped = movies.copy()
movies_mapped['movieId'] = movies_mapped['movieId'].map(map_items)  # usa o mesmo map_items já criado
movies_mapped = movies_mapped.dropna(subset=['movieId']).copy()
movies_mapped['movieId'] = movies_mapped['movieId'].astype(int)

# dicionário {iid_interno: "Genre1|Genre2|..."}
map_genres = dict(zip(movies_mapped['movieId'], movies_mapped['genres']))

# vocabulário de gêneros (V)
all_genres = set()
for g in movies_mapped['genres'].fillna(''):
    for x in g.split('|'):
        x = x.strip()
        if x and x.lower() != '(no genres listed)':
            all_genres.add(x)
V = len(all_genres)

def split_genres(iid):
    """Retorna lista de gêneros para o item interno iid; pode ser []."""
    gstr = map_genres.get(int(iid), '')
    genres = [x for x in gstr.split('|') if x and x.lower() != '(no genres listed)']
    return genres

# 2) Treina contadores por usuário (histórico do treino)
def train_user_model(uid):
    user_hist = train[train.userId == uid][['movieId', 'rating']].copy()
    model = {"relev": Counter(), "n_relev": Counter(), "tot_r": 0, "tot_nr": 0}

    for _, row in user_hist.iterrows():
        genres = split_genres(row.movieId)
        if row.rating >= 3:
            model["relev"].update(genres)
            model["tot_r"] += 1
        else:
            model["n_relev"].update(genres)
            model["tot_nr"] += 1
    return model

# 3) Probabilidade de relevância P(R=1 | gêneros do item) com Laplace (em log-space)
def prob_relevant(uid, iid):
    model = train_user_model(uid)
    genres = split_genres(iid)

    tot_r, tot_nr = model["tot_r"], model["tot_nr"]
    N = tot_r + tot_nr

    # priors com Laplace nos classes (beta(1,1))
    p_r  = (tot_r + 1) / (N + 2)
    p_nr = (tot_nr + 1) / (N + 2)

    # caso extremo: usuário sem histórico → retorna 0.5 (neutro)
    if N == 0:
        return 0.5

    # verossimilhanças com Laplace para cada gênero (Naive Bayes)
    # log P(g|R) = log((count(g,R)+1)/(tot_r + V))
    log_r  = np.log(p_r)
    log_nr = np.log(p_nr)

    for g in genres:
        log_r  += np.log((model["relev"][g]   + 1) / (tot_r  + V))
        log_nr += np.log((model["n_relev"][g] + 1) / (tot_nr + V))

    # converte de volta para probabilidade com softmax de 2 classes
    m = max(log_r, log_nr)  # estabilidade numérica
    pr  = np.exp(log_r  - m)
    pnr = np.exp(log_nr - m)
    return float(pr / (pr + pnr))

def is_relevant(uid, iid, threshold=0.5):
    """Classifica como relevante (True) se prob >= limiar."""
    return prob_relevant(uid, iid) >= threshold

# 4) Exemplo em 5 itens do teste (predição vs rótulo real)
for _, row in test.head(5).iterrows():
    p = prob_relevant(row.userId, row.movieId)
    print(f"user={row.userId:4d} item={row.movieId:4d}  P(relev)={p:.3f}  pred={p>=0.5}  real={row.rating>=3}")

# 5) Avaliação simples no conjunto de teste (acurácia)
y_true = (test['rating'] >= 3).to_numpy()
y_pred = np.array([is_relevant(u, i) for u, i in zip(test.userId, test.movieId)])
acc = (y_true == y_pred).mean()
print(f"Acurácia (teste): {acc:.4f}")

# (Opcional) Métricas adicionais
from sklearn.metrics import precision_recall_fscore_support
prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', zero_division=0)
print(f"Precision: {prec:.4f}  Recall: {rec:.4f}  F1: {f1:.4f}")


user=1836 item=  60  P(relev)=0.931  pred=True  real=False
user=8646 item=  33  P(relev)=0.879  pred=True  real=True
user=1464 item=  19  P(relev)=0.993  pred=True  real=True
user=5315 item=  33  P(relev)=0.497  pred=False  real=True
user=6571 item=  18  P(relev)=0.808  pred=True  real=True
Acurácia (teste): 0.7788
Precision: 0.8136  Recall: 0.9354  F1: 0.8703


***Exercício 04:*** No notebook de exemplos, existe uma implementação de Filtragem Baseada em Conteúdo usando Multi-Layer Perceptron como um regressor (MLPRegressor) que prevê a nota de cada usuário para filmes ainda não vistos. O algoritmo retorna uma lista de top K filmes com maiores notas. O treinamento é realizado utilizando as notas que o usuário deu para os filmes e seus respectivos gêneros.
- Usando a classe MLPClassifier (https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html), implemente uma versão que classifica os filmes não vistos como relevante ou não-relevante.
- No conjunto de dados, realize a binarização das notas, de modo que notas acima de 3 são relevantes e notas abaixo de 3 são não-relevantes.
- Retorne os top k filmes mais relevantes.
- Para um usuário qualquer da base (ou algum outro usuário fictício), analise subjetivamente a qualidade das recomendações, comparando os modelos MLPRegressor e MLPClassifier. 

In [35]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.preprocessing import MultiLabelBinarizer

# ===== 1) Preparar matriz de features por item (gêneros) no ESPAÇO DE IDs INTERNOS =====
movies_mapped = movies.copy()
movies_mapped['movieId'] = movies_mapped['movieId'].map(map_items)  # mapeia p/ ids internos
movies_mapped = movies_mapped.dropna(subset=['movieId']).copy()
movies_mapped['movieId'] = movies_mapped['movieId'].astype(int)

# binarização de gêneros
movies_mapped['genres_split'] = movies_mapped['genres'].fillna('(no genres listed)').apply(lambda x: [g for g in x.split('|') if g and g.lower() != '(no genres listed)'])
mlb = MultiLabelBinarizer()
X_all_items_mlb = mlb.fit_transform(movies_mapped['genres_split'])

# cria uma matriz de features indexada por ID interno (linha = movieId interno)
n_items_total = int(df.movieId.max()) + 1
n_feats = X_all_items_mlb.shape[1]
X_items = np.zeros((n_items_total, n_feats), dtype=np.float32)

id_order = movies_mapped['movieId'].to_numpy()
X_items[id_order] = X_all_items_mlb  # injeta as linhas nas posições corretas

# util: título por id
title_by_id = {int(mid): t for mid, t in zip(movies_mapped['movieId'], movies_mapped['title'])}
def title(iid): return title_by_id.get(int(iid), f"item {int(iid)}")

# ===== 2) Construir dados por usuário =====
def build_user_xy(uid):
    """Retorna X_class, y_class (binário), X_reg, y_reg usando histórico do TREINO."""
    hist = train[train.userId == uid][['movieId', 'rating']]
    if hist.empty:
        return None, None, None, None

    X_list_c = []
    y_list_c = []
    X_list_r = []
    y_list_r = []

    for _, row in hist.iterrows():
        iid = int(row.movieId)
        X_list_c.append(X_items[iid])
        y_list_c.append(1 if row.rating >= 3 else 0)
        X_list_r.append(X_items[iid])
        y_list_r.append(float(row.rating))

    X_class = np.vstack(X_list_c) if X_list_c else None
    y_class = np.array(y_list_c, dtype=int) if y_list_c else None
    X_reg   = np.vstack(X_list_r) if X_list_r else None
    y_reg   = np.array(y_list_r, dtype=float) if y_list_r else None
    return X_class, y_class, X_reg, y_reg

def candidate_items_for_user(uid):
    seen = set(train.loc[train.userId == uid, 'movieId'].tolist())
    all_items = set(range(n_items_total))
    return sorted(list(all_items - seen))

# ===== 3) Treinar modelos por usuário e recomendar =====
def recommend_for_user(uid, k=10, hidden=(64,), random_state=42):
    Xc, yc, Xr, yr = build_user_xy(uid)

    # candidatos (não vistos no TREINO)
    cands = candidate_items_for_user(uid)
    if not cands:
        return [], []

    X_cands = X_items[cands]

    # ---------- Classifier (relevante vs não) ----------
    clf_scores = None
    if Xc is not None and len(np.unique(yc)) >= 2:
        clf = MLPClassifier(
            hidden_layer_sizes=hidden,
            activation='relu',
            solver='adam',
            alpha=1e-4,
            learning_rate_init=1e-3,
            max_iter=300,
            early_stopping=True,
            n_iter_no_change=15,
            random_state=random_state
        )
        clf.fit(Xc, yc)
        clf_scores = clf.predict_proba(X_cands)[:, 1]  # P(relevante)

    # se não der para treinar (poucas amostras ou 1 classe), cai em baseline simples (0.5)
    if clf_scores is None:
        clf_scores = np.full(len(cands), 0.5, dtype=float)

    # Top-K por probabilidade
    topk_idx_clf = np.argsort(clf_scores)[-k:][::-1]
    top_clf = [(int(cands[i]), float(clf_scores[i])) for i in topk_idx_clf]

    # ---------- Regressor (nota prevista) ----------
    reg_scores = None
    if Xr is not None and len(Xr) >= 2:
        reg = MLPRegressor(
            hidden_layer_sizes=hidden,
            activation='relu',
            solver='adam',
            alpha=1e-4,
            learning_rate_init=1e-3,
            max_iter=300,
            early_stopping=True,
            n_iter_no_change=15,
            random_state=random_state
        )
        reg.fit(Xr, yr)
        reg_scores = reg.predict(X_cands)

    # fallback: se não deu para treinar (muito pouco dado), usa média do usuário
    if reg_scores is None:
        user_mean = float(train.loc[train.userId == uid, 'rating'].mean()) if not train.loc[train.userId == uid].empty else 3.0
        reg_scores = np.full(len(cands), user_mean, dtype=float)

    # Top-K por nota prevista
    topk_idx_reg = np.argsort(reg_scores)[-k:][::-1]
    top_reg = [(int(cands[i]), float(reg_scores[i])) for i in topk_idx_reg]

    return top_clf, top_reg

# ===== 4) Rodar para um usuário e imprimir Top-K =====
k = 10
uid = int(train.userId.sample(1, random_state=1).iloc[0])  # escolha um usuário qualquer do treino

top_clf, top_reg = recommend_for_user(uid, k=k, hidden=(64,), random_state=42)

print(f"Usuário {uid} — TOP-{k} (MLPClassifier: prob. de relevância)")
for iid, p in top_clf:
    print(f"  {p:6.3f}  | {title(iid)}")

print(f"\nUsuário {uid} — TOP-{k} (MLPRegressor: nota prevista)")
for iid, s in top_reg:
    print(f"  {s:6.3f}  | {title(iid)}")

# (Opcional) Avaliação simples no conjunto de TESTE — acurácia do classificador em itens do TESTE deste usuário
# Obs.: Como estamos treinando por-usuário, uma avaliação rigorosa exigiria validação cruzada por usuário.
test_uid = test[test.userId == uid]
if not test_uid.empty:
    # previsões do classificador para itens do TESTE do uid
    # (re-treina rapidamente o classificador para o uid; usa a mesma função interna)
    Xc, yc, _, _ = build_user_xy(uid)
    if Xc is not None and len(np.unique(yc)) >= 2:
        clf = MLPClassifier(hidden_layer_sizes=(64,), max_iter=300, early_stopping=True, random_state=42)
        clf.fit(Xc, yc)
        Xi = X_items[test_uid.movieId.astype(int).to_numpy()]
        p = clf.predict_proba(Xi)[:, 1]
        y_true = (test_uid['rating'] >= 3).to_numpy()
        y_pred = (p >= 0.5)
        acc = (y_true == y_pred).mean()
        from sklearn.metrics import precision_recall_fscore_support
        prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', zero_division=0)
        print(f"\nAcurácia no TESTE do usuário {uid}: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f}")
    else:
        print(f"\nUsuário {uid} tem histórico de uma classe só — avaliação do classificador fica indefinida (fallback aplicado).")


Usuário 4926 — TOP-10 (MLPClassifier: prob. de relevância)
   0.654  | Fata Morgana (1971)
   0.645  | River Queen (2005)
   0.645  | Australia (2008)
   0.638  | Heroes of Telemark, The (1965) 
   0.635  | Across the Sea of Time (1995)
   0.631  | Interstellar (2014)
   0.627  | Fist of Fury (Chinese Connection, The) (Jing wu men) (1972)
   0.627  | First on the Moon (Pervye na Lune) (2005)
   0.627  | Mars (1968)
   0.623  | Family, The (Famiglia, La) (1987)

Usuário 4926 — TOP-10 (MLPRegressor: nota prevista)
   4.807  | River Queen (2005)
   4.448  | Alphaville (Alphaville, une étrange aventure de Lemmy Caution) (1965)
   4.258  | Australia (2008)
   4.229  | Balance (1989)
   4.176  | Patema Inverted (2013)
   4.032  | Fist of Fury (Chinese Connection, The) (Jing wu men) (1972)
   4.002  | Cowboy and the Lady, The (1938)
   3.961  | Jack and the Cuckoo-Clock Heart (Jack et la mécanique du coeur) (2013)
   3.902  | Captain America (1990)
   3.893  | 21 (2008)

Acurácia no TESTE do 